In [2]:

import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import matplotlib.cm as cm
import matplotlib as mpl

In [3]:
def read_data(path):
    dsets = []
    for p in path:
        dsets.append(xr.open_dataset(p))
    return xr.merge(dsets)

In [5]:
noresm = read_data(snakemake.input.noresm)
mpi = read_data(snakemake.input.mpiEsm)
ec = read_data(snakemake.input.ecEarth)

In [8]:
mpi_var = mpi['od550aer'].isel(time=slice(5,None)).std(dim='time')
noresm_var = noresm['od550aer'].isel(time=slice(5,None)).std(dim='time')
ec_var = ec['od550aer'].isel(time=slice(5,None)).std(dim='time') 

In [14]:
ec.source_id

In [17]:
fig,ax = plt.subplots(figsize=(7,10), nrows=3, subplot_kw={'projection':ccrs.Robinson()})
cmap = cm.get_cmap('YlOrRd',11)
cmap.set_under('k', alpha=0.0)
mpi_var.plot(ax=ax[0], transform=ccrs.PlateCarree(),vmin=.01, vmax=0.3, add_colorbar=False, cmap=cmap)
ax[0].coastlines()
noresm_var.plot(ax=ax[1], transform=ccrs.PlateCarree(), vmin=.01, vmax=0.3,  cmap=cmap, add_colorbar=False)
ax[1].coastlines()
ec_var.plot(ax=ax[2], transform=ccrs.PlateCarree(), vmin=.01, vmax=0.3,  cmap=cmap, add_colorbar=False)

ax[2].coastlines()
ax[0].set_title('MPI-ESM-1-2-HAM')
ax[1].set_title('NorESM2-LM')
ax[2].set_title('EC-Earth3-AerChem')
cax = fig.add_axes([0.3,0.06,0.4,0.02])
cb = fig.colorbar(cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=0.01, vmax=0.3), cmap=cmap), cax=cax, 
                 orientation='horizontal', extend = 'min')
fig.suptitle(snakemake.params.fig_title, fontsize=16,y=0.95)
plt.savefig(snakemake.output.aod_varibility_map, dpi=144, bbox_inches='tight')